## Making test vocabulary and finding features

In [151]:
import os
import numpy as np
x = []   #x is the list where 1st element is name of document and second element is text in document
y = []   # y is the category
for category in os.listdir("20_newsgroups"):
    for document in os.listdir("20_newsgroups/" + category):
        with open("20_newsgroups/" + category + '/' + document, "r") as f:
            x.append((document,f.read()))
            y.append(category)

In [152]:
# Making test vocabulary
words = {}
stop_words = {"ourselves", "hers", "between", "yourself", "but", 
              "again", "there", "about", "once", "during", "out", 
              "very", "having", "with", "they", "own", "an", "be", 
              "some", "for", "do", "its", "yours", "such", "into", 
              "of", "most", "itself", "other", "off", "is", "s", 
              "am", "or", "who", "as", "from", "him", "each", "the",
              "themselves", "until", "below", "are", "we", "these",
              "your", "his", "through", "don", "nor", "me", "were",
              "her", "more", "himself", "this", "down", "should", "our",
              "their", "while", "above", "both", "up", "to", "ours",
              "had", "she", "all", "no", "when", "at", "any", "before",
              "them", "same", "and", "been", "have", "in", "will", "on",
              "does", "yourselves", "then", "that", "because", "what", "over",
              "why", "so", "can", "did", "not", "now", "under", "he", "you",
              "herself", "has", "just", "where", "too", "only", "myself",
              "which", "those", "i", "after", "few", "whom", "t", "being",
              "if", "theirs", "my", "against", "a", "by", "doing", "it",
              "how", "further", "was", "here", "than"}

for i in range(len(x)):
    for word in x[i][1].split():
        if word.lower() in stop_words:
            continue
        elif word.lower() in words:
            words[word.lower()] += 1
        else:
            words[word.lower()] = 1

In [153]:
# Function to get k most frequent words from vocabulary 
def get_top_k(words, k):
    freq = np.array(list(words.values()))
    freq = np.sort(freq, axis = -1, kind = "quicksort", order = None)[::-1]
    new_words = {key : val for key, val in words.items() if val >= freq[k - 1]}
    return new_words

In [154]:
# Mapping top frequnecy words to an index value
top_k_words = get_top_k(words, 1000)
i = 0
for word in top_k_words.keys():
    top_k_words[word] = i
    i += 1

In [155]:
# Making final 2D array with top words as features to begin algorithm implementation
rows, cols = (len(x), len(top_k_words)) 
final_x = [[0 for i in range(cols)] for j in range(rows)]

for i in range(len(x)):
    for word in x[i][1].split():
        if word.lower() in top_k_words:
            final_x[i][top_k_words[word.lower()]] += 1

In [156]:
# Converting lists to numpy arrays for easier manipulation
final_x = np.array(final_x)
final_y = np.array(y)

## Own implementation of Naive Bayes

In [157]:
# Getting the train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(final_x, final_y)

In [159]:
# Function to get the count dictionary
def get_count_dict(X_train, Y_train):
    count = {}
    target_names = set(Y_train)
    
    for target in target_names:
        X_subset = X_train[Y_train == target]
        count[target] = {}
        count[target]["total"] = 0
        column_sum = np.sum(X_subset, axis = 0)
        for i in range(X_train.shape[1]):
            count[target][i] = column_sum[i]
            count[target]["total"] += column_sum[i]
            
    return count

In [160]:
count = get_count_dict(X_train, Y_train)

In [161]:
# Algorithm implementation for a single document
def predict_for_one(X_test_row):
    
    target_names = set(Y_train)
    max_probability = 0
    first_run = True
    predicted_target = ""
    
    for target in target_names:
        
        target_probability = 0
        
        for i in range(X_train.shape[1]):
            num = count[target][i] + 1
            den = count[target]["total"] + X_train.shape[1]
            target_probability += (X_test_row[i] * np.log2(num / den))
            
        if (target_probability > max_probability) or (first_run):
            max_probability = target_probability
            predicted_target = target
            first_run = False
            
    return predicted_target

In [162]:
# Algorithm implementation for all test documents
def multi_naive_bayes():
    Y_pred = []
    
    for i in range(X_test.shape[0]):
        Y_pred.append(predict_for_one(X_test[i]))
        
    Y_pred = np.array(Y_pred)
    return Y_pred

In [164]:
Y_pred = multi_naive_bayes()

## Inbuilt implementation of Naive Bayes

In [165]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)
Y_pred_inbuilt = clf.predict(X_test)

## Comparison between the two implementations

In [166]:
from sklearn.metrics import classification_report
# Classification report for own implementation
print(classification_report(Y_test, Y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.68      0.71      0.70       263
           comp.graphics       0.72      0.71      0.71       246
 comp.os.ms-windows.misc       0.84      0.74      0.79       237
comp.sys.ibm.pc.hardware       0.84      0.80      0.82       256
   comp.sys.mac.hardware       0.80      0.91      0.85       235
          comp.windows.x       0.90      0.68      0.78       246
            misc.forsale       0.58      0.91      0.71       256
               rec.autos       0.68      0.81      0.74       244
         rec.motorcycles       0.73      0.89      0.80       236
      rec.sport.baseball       0.75      0.91      0.82       249
        rec.sport.hockey       0.94      0.62      0.75       256
               sci.crypt       0.92      0.82      0.87       248
         sci.electronics       0.72      0.82      0.76       262
                 sci.med       0.71      0.79      0.75       230
         

In [167]:
# Classification report for inbuilt implementation
print(classification_report(Y_test, Y_pred_inbuilt))

                          precision    recall  f1-score   support

             alt.atheism       0.68      0.71      0.70       263
           comp.graphics       0.72      0.71      0.71       246
 comp.os.ms-windows.misc       0.84      0.75      0.79       237
comp.sys.ibm.pc.hardware       0.84      0.80      0.82       256
   comp.sys.mac.hardware       0.80      0.91      0.85       235
          comp.windows.x       0.90      0.68      0.78       246
            misc.forsale       0.59      0.91      0.71       256
               rec.autos       0.68      0.81      0.74       244
         rec.motorcycles       0.73      0.89      0.80       236
      rec.sport.baseball       0.76      0.91      0.83       249
        rec.sport.hockey       0.95      0.62      0.75       256
               sci.crypt       0.92      0.82      0.87       248
         sci.electronics       0.72      0.82      0.77       262
                 sci.med       0.71      0.79      0.75       230
         